In [3]:
import os
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense
from tensorflow.keras.layers import Dropout

trainDataGen = ImageDataGenerator(rotation_range = 5,
                                  width_shift_range = 0.1,
                                  height_shift_range = 0.1,
                                  rescale = 1.0/255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = False,
                                  fill_mode = 'nearest')

testDataGen = ImageDataGenerator(rescale = 1.0/255)

trainGenerator = trainDataGen.flow_from_directory(os.path.join("Splitted_Dataset", "Train"),
                                                  target_size = (32,32),
                                                  batch_size = 32,
                                                  color_mode = "grayscale",
                                                  classes = [str(class_id) for class_id in range(49)],
                                                  class_mode = "categorical")

validationGenerator = testDataGen.flow_from_directory(os.path.join("Splitted_Dataset", "Validation"),
                                                      target_size = (32,32),
                                                      batch_size = 32,
                                                      color_mode = "grayscale",
                                                      classes = [str(class_id) for class_id in range(49)],
                                                      class_mode = "categorical")

model = Sequential()

model.add(Conv2D(32, (3, 3), strides = 1, activation = "relu", input_shape = (32, 32, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2), strides = (2, 2), padding = "same"))

model.add(Conv2D(32, (3, 3), strides = 1, activation = "relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2), strides = (2, 2), padding = "same"))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), strides = 1, activation = "relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2), strides = (2, 2), padding = "same"))

model.add(Flatten())
model.add(Dense(256, activation="relu", kernel_initializer='he_uniform'))
model.add(Dropout(0.55))

model.add(Dense(100, activation="relu", kernel_initializer='he_uniform'))
model.add(Dropout(0.35))

model.add(Dense(49, activation="softmax"))

model.compile(optimizer = Adam(lr = 1e-3, decay = 1e-5), loss = "categorical_crossentropy", metrics = ['accuracy'])

callbacks = [ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1,
                              patience = 7, min_lr = 1e-5),
             EarlyStopping(patience = 9, # Patience should be larger than the one in ReduceLROnPlateau
                          min_delta = 1e-5),
             CSVLogger("training.log", append = True),
             ModelCheckpoint('backup_last_model.hdf5'),
             ModelCheckpoint('best_val_acc.hdf5', monitor = 'val_accuracy', mode = 'max', save_best_only = True),
             ModelCheckpoint('best_val_loss.hdf5', monitor = 'val_loss', mode = 'min', save_best_only = True)]

model.fit(trainGenerator, epochs = 50, validation_data = validationGenerator, callbacks = callbacks)

model = load_model('best_val_acc.hdf5')
loss, acc = model.evaluate(validationGenerator)

print("Best Accuracy Model:")
print('Loss on Validation Data : ', loss)
print('Accuracy on Validation Data :', '{:.4%}'.format(acc))

model = load_model('best_val_loss.hdf5')
loss, acc = model.evaluate(validationGenerator)

print("Best Loss Model:")
print('Loss on Validation Data : ', loss)
print('Accuracy on Validation Data :', '{:.4%}'.format(acc))


Found 79425 images belonging to 49 classes.
Found 8825 images belonging to 49 classes.
Epoch 1/50
2483/2483 [==============================] - 60s 24ms/step - loss: 2.1228 - accuracy: 0.3977 - val_loss: 0.7614 - val_accuracy: 0.7866
Epoch 2/50
2483/2483 [==============================] - 47s 19ms/step - loss: 0.9423 - accuracy: 0.7047 - val_loss: 0.6645 - val_accuracy: 0.8146
Epoch 3/50
2483/2483 [==============================] - 47s 19ms/step - loss: 0.7040 - accuracy: 0.7800 - val_loss: 0.3105 - val_accuracy: 0.9018
Epoch 4/50
2483/2483 [==============================] - 47s 19ms/step - loss: 0.5953 - accuracy: 0.8139 - val_loss: 0.2386 - val_accuracy: 0.9229
Epoch 5/50
2483/2483 [==============================] - 46s 18ms/step - loss: 0.5270 - accuracy: 0.8371 - val_loss: 0.1861 - val_accuracy: 0.9328
Epoch 6/50
2483/2483 [==============================] - 47s 19ms/step - loss: 0.4777 - accuracy: 0.8531 - val_loss: 0.1665 - val_accuracy: 0.9453
Epoch 7/50
2483/2483 [===============